INFORMATION RETRIEVAL

PRANSHU PARATE

202211063


In [1]:
#installing the python-terrier library
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [2]:
#import the PyTerrier library for information retrieval tasks
import pyterrier as pt

if not pt.started():
  pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [9]:
#import evaluation measures from the PyTerrier library
from pyterrier.measures import *

In [3]:
#load and get the dataset
df = pt.get_dataset("trec-deep-learning-docs")

In [4]:
#define the fields to be indexed
columns = ["title", "body"]

#create a PyTerrier TRECCollectionIndexer for indexing
indexer = pt.TRECCollectionIndexer("./field_indexing")

#set properties for the indexer to specify how fields should be processed
pt.ApplicationSetup.setProperty("indexer.meta.forward.keys", ",".join(columns))
pt.ApplicationSetup.setProperty("indexer.meta.reverse.keys", ",".join(columns))
pt.ApplicationSetup.setProperty("FieldTags.process", ",".join(columns))
pt.ApplicationSetup.setProperty("TrecDocTags.doctag", "DOC")
pt.ApplicationSetup.setProperty("TrecDocTags.idtag", "DOCNO")
pt.ApplicationSetup.setProperty("TrecDocTags.skip", "DOCHDR")

#indexing with the fields
indexref = indexer.index(df.get_corpus())


msmarco-docs.trec.gz:   0%|          | 0.00/7.92G [00:00<?, ?iB/s]

In [5]:
#indexing with the fields
doc_index = pt.IndexFactory.of(indexref)

In [6]:
#loading "topic" and "qrel" document
topic = df.get_topics("test")
qrel = df.get_qrels("test")

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/331k [00:00<?, ?iB/s]

In [7]:
# Define the models tfidf, bm25, pl2, bm25f, pl2f, dph
tf_idf = pt.BatchRetrieve(doc_index, wmodel="TF_IDF") % 100
bm25 = pt.BatchRetrieve(doc_index, wmodel="BM25")% 100
pl2 = pt.BatchRetrieve(doc_index, wmodel="PL2") % 100
bm25f = pt.BatchRetrieve(doc_index, wmodel="BM25F")% 100
pl2f = pt.BatchRetrieve(doc_index, wmodel="PL2F") % 100
dph = pt.BatchRetrieve(doc_index, wmodel="DPH") % 100

In [10]:
#metrics used for evaluation
eval_metric = [RR@10,RR@100,AP@10, AP@100,nDCG@10, nDCG@100]

In [11]:
#evaluate the performance of several retrieval models
pt.Experiment(
    [tf_idf,bm25,pl2,bm25f,pl2f,dph],
    topic,
    qrel,
    eval_metric
)

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,"RankCutoff(BR(TF_IDF), 100)",0.864987,0.866044,0.113309,0.244909,0.524194,0.503096
1,"RankCutoff(BR(BM25), 100)",0.874123,0.874123,0.114305,0.248334,0.538802,0.512328
2,"RankCutoff(BR(PL2), 100)",0.841085,0.842746,0.112509,0.248587,0.542381,0.518796
3,"RankCutoff(BR(BM25F), 100)",0.011628,0.011628,0.023256,0.023256,0.018011,0.018011
4,"RankCutoff(BR(PL2F), 100)",0.011628,0.012528,0.023256,0.023258,0.018011,0.018190
5,"RankCutoff(BR(DPH), 100)",0.907946,0.907946,0.115437,0.249372,0.543129,0.515971


I observed that:

Highest RR values at both Rank 10 and Rank 100 are achieved by the "BM25" model, followed closely by "DPH."

"BM25" performs the best, having the highest AP values at both Rank 10 and Rank 100. "DPH" follows as the second-best model in terms of AP.

Similar to RR and AP, "BM25" and "DPH" achieve the highest nDCG values at both Rank 10 and Rank 100.

BM25F" and "PL2F" models have very low RR, AP, and nDCG values at both rank 10 and 100, it means that they may not perform well on the given dataset.

"BM25" and "DPH" appear to be the top-performing models for this specific dataset and evaluation criteria.